# Scraping with Beautiful Soup

After eating chocolate bars your whole life, you've decided to go on a quest to find the greatest chocolate bar in the world.

You've found a website that has a table with over 1700 reviews of chocolate bars from all around the world.

The rating scale is from 1-5, as described in <a href="http://flavorsofcacao.com/review_guide.html">this review guide</a>. A `1` is "unpleasant" chocolate, while a `5` is a bar that transcends "beyond the ordinary limits".

## Make Some Chocolate Soup

1. Let's make a request to the website to get the raw HTML, which we later turn into a BeautifulSoup object.

   The URL is

```
https://content.codecademy.com/courses/beautifulsoup/cacao/index.html
```

In [ ]:
import requests
chocolate = requests.get("https://content.codecademy.com/courses/beautifulsoup/cacao/index.html")

2. Create a BeautifulSoup object called `soup` to traverse this HTML.

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(chocolate.content, "html.parser")

3. Print out the `soup` object to explore the HTML.

In [ ]:
print(soup)

## Where are the best cocao beans grown?

4. How are ratings distributed? Let's make a histogram of this data.

In [ ]:
rating_tags = soup.find_all(attrs={"class": "Rating"})
rating_tags

5. Create a list to store all the ratings in.

In [ ]:
ratings = []

In [ ]:
for rating in rating_tags[1:]:
    rate_text = rating.get_text()
    rate_score = float(rate_text)
    ratings.append(rate_score)
ratings

6. Create a histogram of the ratings values:

In [ ]:
import matplotlib.pyplot as plt
plt.hist(ratings)
plt.show()

## Which chocolatier makes the best chocolate?

7. We want to now find the 10 most highly rated chocolatiers.
 
   Create a DataFrame that has the chocolate companies in one column, and the ratings in another. Then, find the ones with the highest average rating.

In [ ]:
company_tags = soup.select(".Company")
company_tags

In [ ]:
names = []

In [ ]:
for td in company_tags[1:]:
    names.append(td.get_text())

In [ ]:
import pandas as pd
company_ratings = {"Company": names, "Ratings": ratings}
cacao_df = pd.DataFrame.from_dict(company_ratings)

8. Group your DataFrame by Company and take the average of the grouped ratings.

    Then, get the 10 highest rated chocolate companies.

In [ ]:
mean_ratings = cacao_df.groupby("Company").Ratings.mean()
ten_best = mean_ratings.nlargest(10)
print(ten_best)

## Is more cacao better?

9. We want to see if the chocolate experts tend to rate chocolate bars with higher levels of cacoa to be better than those with lower levels of cacoa.
   
   Create a list that contains all of the cocoa percentages. 

In [ ]:
cocoa_percents = []
cocoa_percent_tags = soup.select(".CocoaPercent")

for td in cocoa_percent_tags[1:]:
    percent = float(td.get_text().strip('%'))
    cocoa_percents.append(percent)

10. Add the cocoa percentages as a column called `"CocoaPercentage"` in the DataFrame that has companies and ratings in it.

In [ ]:
cocoa = {"Company": names, "Ratings": ratings, "CocoaPercentage": cocoa_percents}
cocoa_df = pd.DataFrame.from_dict(cocoa)

11. Make a scatterplot of ratings vs percentage of cocoa.

In [ ]:
plt.scatter(cocoa_df.CocoaPercentage, cocoa_df.Ratings)
plt.show()

12. Draw a line of best-fit over the scatterplot.

In [ ]:
import numpy as np
plt.scatter(cocoa_df.CocoaPercentage, cocoa_df.Ratings)
z = np.polyfit(cocoa_df.CocoaPercentage, cocoa_df.Ratings, 1)
line_function = np.poly1d(z)
plt.plot(cocoa_df.CocoaPercentage, line_function(cocoa_df.CocoaPercentage), "r-")
plt.show()

## Explore!

13. What other kinds of questions can you answer? Use a combination of BeautifulSoup and Pandas to explore some more.

In [ ]:
## Question 1: Where are the best cocoa beans grown?
origins = []
origin_tags = soup.select(".BroadBeanOrigin")

for td in origin_tags[1:]:
    country = td.get_text()
    origins.append(country)
    
beans = {"Bean Origin": origins, "CocoaPercentage": cocoa_percents}
beans_df = pd.DataFrame.from_dict(beans)

mean_percent = beans_df.groupby("Bean Origin").CocoaPercentage.mean()
ten_best = mean_percent.nlargest(10)
print(ten_best)

In [ ]:
## Question 2: Which countries produce the highest-rated bars?
countries = []
country_tags = soup.select(".CompanyLocation")

for td in country_tags[1:]:
    country = td.get_text()
    countries.append(country)
    
bars = {"CompanyLocation": countries, "Rating": ratings}
bars_df = pd.DataFrame.from_dict(bars)

mean_rating = bars_df.groupby("CompanyLocation").Rating.mean()
ten_best = mean_rating.nlargest(10)
print(ten_best)